In [ ]:
from autumn.infrastructure.remote import springboard
from autumn.projects.sm_covid2.common_school.runner_tools import (
    run_full_analysis, 
    print_continuous_status, 
    download_analysis,
    DEFAULT_RUN_CONFIG,
    TEST_RUN_CONFIG,
)

In [ ]:
DEFAULT_RUN_CONFIG

In [ ]:
run_config = DEFAULT_RUN_CONFIG  # update if required

In [ ]:
ISO3_LIST = ["AUS", "BOL", "FRA"]
ANALYSIS = "main"

In [ ]:
def remote_full_analysis_task(bridge: springboard.task.TaskBridge, iso3: str = 'FRA', analysis: str = "main"):
    
    import multiprocessing as mp
    mp.set_start_method('forkserver')

    bridge.logger.info(f"Running full analysis for {iso3}. Analysis type: {analysis}.")

    idata, uncertainty_df, diff_quantiles_df = run_full_analysis(
        iso3,
        run_config=run_config,
        output_folder=bridge.out_path,
        logger=bridge.logger
    )
   
    bridge.logger.info("Full analysis complete")

In [ ]:
mspec = springboard.EC2MachineSpec(run_config['n_cores'], 4, "compute")

In [ ]:
analysis_title = "test_multirun"
config_str = f"_{ANALYSIS}_LHS{run_config['n_opti_searches']}_opt{run_config['opti_budget']}_mc{run_config['metropolis_tune']}n{run_config['metropolis_draws']}"
run_paths = [springboard.launch.get_autumn_project_run_path("school_project", iso3, analysis_title + config_str) for iso3 in ISO3_LIST]
run_paths

In [ ]:
run_group_name = run_paths[0].split("/")[-1]
run_group_name

In [ ]:
task_dict = {}
for i, iso3 in enumerate(ISO3_LIST):
    # Use these targets, but keep the other kwargs the same
    task_kwargs = {
        "iso3": iso3,
        "analysis": ANALYSIS
    }
    task_dict[run_paths[i]] = springboard.TaskSpec(remote_full_analysis_task, task_kwargs)

In [ ]:
runners = springboard.launch.launch_synced_multiple_autumn_task(task_dict, mspec, run_group=run_group_name)

In [ ]:
for run_path, runner in runners.items():
    print(runner.s3.get_status())
    # print(runner.top("%MEM"))

In [ ]:
for run_path in run_paths:
    download_analysis(run_path)